In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import re
import csv
from sklearn.metrics import confusion_matrix

In [2]:
# Header information
headers = ['User_ID,', "Gender", "Age", "Occupation", "Star_Sign", "Date", "Text"]

In [3]:
# Preprocessing the information

def preprocess_X(data):
    
    X = data[:,6]
    return fix_string_array(X)

def preprocess_y(data):
    y = data[:,2]
    return y

def remove_junk_text(text):
    text = (str)(text)
    text = str.lower(text)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    return text


# removing all the junk text in for a given array
def fix_string_array(arr):
    
    for i in range(arr.shape[0]):
        arr[i] = remove_junk_text((str)(arr[i]))
    
    return arr
        
#X_test[:,6] = X_test[:,6].lower() #remove_junk_text(X_test[:,6])

In [4]:
def get_distinct_col_vals(lst, col_num):
    new_lst = []
    for row in lst:
        new_lst.append(row[col_num])
    return get_distinct_vals(new_lst)

def get_distinct_vals(list1):
    list1 = set(list1)
    return list1


#get_distinct_col_vals(data, 2)


In [5]:
# reading in file names etc.
PATH = "proj_data/"

training_data = pd.read_csv(PATH + "train_raw.csv")
training_data = training_data.values

test_data = pd.read_csv(PATH + "dev_raw.csv")
test_data = test_data.values

X_train = (preprocess_X(training_data))  # text vals
y_train = preprocess_y(training_data)  # the age values
y_train = y_train.astype(int)

X_test = preprocess_X(test_data)
y_test = preprocess_y(test_data)
y_test = y_test.astype(int)



In [6]:
# Initialising the datasets
# for TOP 10 data

training_top10_data = pd.read_csv(PATH + "train_top10.csv")
training_top10_data = training_top10_data.values

test_top10_data = pd.read_csv(PATH + "dev_top10.csv")
test_top10_data = test_top10_data.values

X_top10_train = training_top10_data[:,1:31]
y_top10_train = training_top10_data[:, 31]

X_top10_test = test_top10_data[:,1:31]
y_top10_test = test_top10_data[:, 31]

In [7]:
# for final test data
FINAL_PATH = "COMP30027_2018S1_proj2-test/"
final_raw = FINAL_PATH + "test_raw.csv"
final_top10 = FINAL_PATH + "test_top10.csv"


final_data_raw = pd.read_csv(final_raw)
final_data_top10 = pd.read_csv(final_top10)

final_raw = final_data_raw.values
final_top10 = final_data_top10.values

X_final_raw = preprocess_X(final_raw)
y_final_raw = preprocess_y(final_raw)


X_final_top10 = final_top10[:,1:31]
y_final_top10 = final_top10[:, 31]
#X_final_top10 = 

In [9]:
# Creating a dictionary for the terms
from sklearn.feature_extraction.text import CountVectorizer
vectoriser = CountVectorizer()
X_train_dict = vectoriser.fit_transform(X_train)
X_test_dict = vectoriser.transform(X_test)


In [10]:
# Used to transform the values to TF-IDF features.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_dict = tfidf_transformer.fit_transform(X_train_dict)
X_train_dict.shape

(276414, 705820)

In [28]:
# create vectorisers:
# Creating a bag of words using a training and test set
# Takes input of a cleaneded db containing text for a training and test set
def createBagOfWords(train, test):
    vectoriser = CountVectorizer()
    bagged_train = vectoriser.fit_transform(train)
    bagged_test = vectoriser.fit_transform(test)
    
    tfidf_transformer = TfidfTransformer()
    bagged_train = tfidf_transformer.fit_transform(bagged_train)
    bagged_test = tfidf_transformer.fit_transform(bagged_test)
    
    return (bagged_train, bagged_test)
    

In [31]:
from sklearn.naive_bayes import MultinomialNB
def do_NB(train, test):
    MultiNB = MultinomialNB()
    return MultinomialNB().fit(train, test)

In [35]:
from sklearn.metrics import accuracy_score
def check_accuracy(classifier, testX, testY):
    predicted = txt_clf.predict(testX)
    print("Exactly correct ->", accuracy_score(predicted, testY))
    print("Close at +-3 ->", np.mean(abs(predicted - testY) < 3))
    print("Not great at +-5 ->", np.mean(abs(predicted - testY) < 5))
    

In [39]:
# Creating a simple base classifier - Naive Bayes:

#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
X_train_NB, X_test_NB = createBagOfWords(X_train, X_test)

NB_classifier = do_NB(X_train_NB, y_train)
check_accuracy(NB_classifier, X_test_NB, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [276414, 45331]

In [ ]:
# Analysing NB:
check_accuracy(NB_classifier, X_train, y_test)

# Error analysis
confusion_matrix(predicted, y_test)


In [ ]:
# Created pipeline with help from: 
# https://blog.sicara.com/naive-bayes-classifier-sklearn-python-example-tips-42d100429e44

from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

txt_clf = text_clf.fit(X_train, y_train)

In [14]:
from sklearn.metrics import accuracy_score
# How often it was right on the money about the age
predicted = txt_clf.predict(X_test)
accuracy_score(predicted,y_test)

0.19714985330127285

In [15]:
# Error analysis
confusion_matrix(predicted, y_test)


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   0,    9,    4,    0,    0,    2,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   0,   33,   26,   51,    3,    2,    2,    1,    1,    2,    1,
           0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 110,  957, 1030, 2140,  998,   96,  275,  129,  125,   50,   21,
           5,   11,    3,    1,    0,    5,    5,    1,    4,    0,    0,
           2,    7,    4,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  

In [16]:
# Were asked to put the data into the given 
def transform_age(data):
    data[data<14] = 0 # question mark!
    data[(data>16)&(data<24)] = 0
    data[(data>26)&(data<34)] = 0
    data[(data>36)&(data<44)] = 0  
    data[data>46] = 0
    
    data[data==14] = 14
    data[data==15] = 14
    data[data==16] = 14
    data[data==24] = 24
    data[data==25] = 24
    data[data==26] = 24
    data[data==34] = 34
    data[data==35] = 34
    data[data==36] = 34
    data[data==44] = 44
    data[data==45] = 44
    data[data==46] = 46
    

In [17]:
# transform into a for more similar to final
pred = predicted
y = y_test
transform_age(pred)
transform_age(y)

In [18]:
# Prediciting how many words were in the correct ranges
predicted = text_clf.predict(X_test)
accuracy_score(pred,y)

0.46356797776356135

In [19]:
# Error analysis
confusion_matrix(predicted, y_test)


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    2,    13,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    8,   110,     0,     0,     4,     0,     0,     1,     0,
            0],
       [ 1295,  4127,     0,     0,   529,     0,     0,    19,     2,
            7],
       [10361,  8769,     0,     0, 16482,     0,     0,  2547,   429,
          101],
       [  105,    68,     0,     0,   222,     0,     0,    15,     7,
            2],
       [   28,    13,     0,     0,    60,     0,     0,     2,     2,
            1],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]])

In [20]:
# How often it was close enough:
# Is able to tell within a range of 3 years either side.
predicted = text_clf.predict(X_test)
np.mean(abs(predicted - y_test) < 3)

0.46356797776356135

In [21]:
# Using support vector machines!
# Training Support Vector Machines - SVM and calculating its performance

# Using it on the ? values

from sklearn.linear_model import SGDClassifier
svm_classifier = Pipeline([('clf-svm', SGDClassifier(n_iter=10))])

svm_classifier = svm_classifier.fit(X_top10_train, y_top10_train)
svm_classifier = svm_classifier.predict(X_top10_test)
accuracy_score(predicted_svm, y_top10_test)



/Users/chris/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


NameError: name 'predicted_svm' is not defined

In [22]:
# Using support vector machines!
# Training Support Vector Machines - SVM and calculating its performance


from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('clf-svm', SGDClassifier(loss='hinge', penalty='l1',alpha=1e-3, n_iter=10, random_state=42))])

text_clf_svm = text_clf_svm.fit(X_top10_train, y_top10_train)
predicted_svm = text_clf_svm.predict(X_top10_test)
accuracy_score(predicted_svm,y_top10_test)

/Users/chris/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.41916128036001854

In [23]:
# Training Support Vector Machines - SVM and calculating its performance

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)


/Users/chris/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.14449273124351988

In [24]:
# The numbers that were close enough
predicted_svm = text_clf.predict(X_test)
np.mean(abs(predicted_svm - y_test) < 3)

0.46356797776356135

In [25]:
# Error analysis
confusion_matrix(predicted_svm, y_test)


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    2,    13,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    8,   110,     0,     0,     4,     0,     0,     1,     0,
            0],
       [ 1295,  4127,     0,     0,   529,     0,     0,    19,     2,
            7],
       [10361,  8769,     0,     0, 16482,     0,     0,  2547,   429,
          101],
       [  105,    68,     0,     0,   222,     0,     0,    15,     7,
            2],
       [   28,    13,     0,     0,    60,     0,     0,     2,     2,
            1],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]])

In [28]:
# K nearest neighbours
# I know this isn't going to be strong classifier, especially as slow as it is, but I was curious
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_top10_train, y_top10_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
predicted = neigh.predict(X_top10_test)
#predicted
np.mean(predicted == y_top10_test)

0.25421896715272108

In [31]:
confusion_matrix(predicted, y_top10_test)

array([[2382, 1137,   65,   16, 1005],
       [4459, 8209, 1585,  280, 5035],
       [6256, 7946,  932,  254, 5755],
       [   3,    5,    2,    1,    4],
       [   0,    0,    0,    0,    0]])

In [ ]:
predicted_knn = neigh.predict(X_top10_test)
np.mean(abs(predicted_knn - y_top10_test) < 3)

In [ ]:
from sklearn.metrics import confusion_matrix
y_top10_test[0]
confusion_matrix(predicted, y_top10_test, labels=["14-16", "24-26", "34-36", "44-46", "?"]) #,labels=["ant", "bird", "cat"])

0.254218967153
0.401932452406
0.401579493062
0.410866735788


In [ ]:
# Holy moly is above slow! Used 7

In [7]:
X_top10_train[0:100]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [0, 0, 0, ..., 3, 0, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=object)

In [13]:
x = X_top10_train[0:500]
y = y_top10_train[0:500]

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC()
svc_classifier.fit(X_top10_train, y_top10_train)

In [ ]:
predicted = svc_classifier.predict(X_top10_test)
np.mean(predicted == y_top10_test)

In [ ]:
print(confusion_matrix(predicted, y_top10_test, labels=["14-16", "24-26", "34-36", "44-46", "?"]))

In [7]:
len(X_top10_test)

45331

In [12]:
# Choosing k based off standard
k = (len(X_top10_test))**(1/2)

212.91077943589423

In [18]:
# K nearest neighbours

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=k)
neigh.fit(X_top10_train, y_top10_train)
predicted = neigh.predict(X_top10_test)
#predicted
print(np.mean(predicted == y_top10_test))


0.275330347886


In [ ]:
for k in range(100, 300):

    from sklearn.neighbors import KNeighborsClassifier
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_top10_train, y_top10_train)
    predicted = neigh.predict(X_top10_test)
    #predicted
    print(k, np.mean(predicted == y_top10_test))

In [8]:
# FINAL PREDICTION
# Uses K nearest neighbours that I found to perform the best
# The predictor

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=101)
neigh.fit(X_top10_train, y_top10_train)
predicted = neigh.predict(X_final_top10)
#predicted
print(predicted)


['24-26' '24-26' '24-26' ..., '24-26' '24-26' '24-26']


In [9]:
f= open("prediction.txt","w+")
for i in predicted:
     f.write(i)
f.close()





In [ ]:
#Linear Regression

